# Import packages

In [4]:
import pandas as pd
from teradataml import *
import ast
import json
import torch
from teradataml.dataframe.copy_to import copy_to_sql
from transformers.generation import GenerationMixin

from modules.db import TeradataDatabase
from modules.models import( 
    OpusTranslationModelConfig, 
                        OpusTranslationModel, 
                        SentenceEmbeddingConfig, 
                        SentenceEmbeddingModel, 
)
from utils import clean_text, load_embedding_model, unicode_clean, load_translation_model
from constants import (
     GPC_PATH
)


## Connect to database

In [5]:
td_db = TeradataDatabase()
td_db.connect()

In [ ]:
tdf = td_db.execute_query("Select * from demo_user.gpc_orig")
df = pd.DataFrame(tdf)


In [ ]:
df.head()

### Remvoing un-neccesary columns

In [ ]:
query = """
ALTER TABLE demo_user.gpc_orig 
DROP SegmentCode,
DROP FamilyCode,
DROP ClassCode,
Drop BrickCode,
DROP AttributeCode,
DROP AttributeTitle,
DROP AttributeDefinition,
DROP AttributeValueCode,
DROP AttributeValueTitle,
DROP AttributeValueDefinition;
"""

In [11]:
tdf = td_db.execute_query(query)

### Cleaning the GPC Columns

In [ ]:
cleaning_query = """
UPDATE demo_user.gpc_orig
SET SegmentTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    SegmentDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(SegmentDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    FamilyDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(FamilyDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    ClassDefinition = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(ClassDefinition, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickTitle = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickTitle, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Includes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Includes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' '))),
    BrickDefinition_Excludes = LOWER(TRIM(
                          REGEXP_REPLACE(
                            REGEXP_REPLACE(
                              REGEXP_REPLACE(BrickDefinition_Excludes, '[[:digit:]]+', ''), 
                              '[-_/\\|]', ' '),                              
                            '[[:punct:]]', ' ')));

"""

In [13]:
tdf = td_db.execute_query(cleaning_query)

## Disconnect

In [ ]:
td_db.disconnect()

In [6]:
gpc_df = pd.read_excel(GPC_PATH)
gpc_df.head()

,SegmentCode,SegmentTitle,SegmentDefinition,FamilyCode,FamilyTitle,FamilyDefinition,ClassCode,ClassTitle,ClassDefinition,BrickCode,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes,AttributeCode,AttributeTitle,AttributeDefinition,AttributeValueCode,AttributeValueTitle,AttributeValueDefinition
0,70000000,Arts/Crafts/Needlework,All products that are used to any creative act...,70010000,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,70010100,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,10001674,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...,20001337.0,Type of Artists Brush/Applicator,"Indicates, with reference to the product brand...",30008491.0,ARTISTS BLENDER/FINISHING BRUSH,NaN
1,70000000,Arts/Crafts/Needlework,All products that are used to any creative act...,70010000,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,70010100,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,10001674,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...,20001337.0,Type of Artists Brush/Applicator,"Indicates, with reference to the product brand...",30008492.0,ARTISTS CALLIGRAPHY BRUSH,NaN
2,70000000,Arts/Crafts/Needlework,All products that are used to any creative act...,70010000,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,70010100,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,10001674,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...,20001337.0,Type of Artists Brush/Applicator,"Indicates, with reference to the product brand...",30008493.0,ARTISTS FLAT BRUSH,NaN
3,70000000,Arts/Crafts/Needlework,All products that are used to any creative act...,70010000,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,70010100,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,10001674,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...,20001337.0,Type of Artists Brush/Applicator,"Indicates, with reference to the product brand...",30008494.0,ARTISTS ROUND BRUSH,NaN
4,70000000,Arts/Crafts/Needlework,All products that are used to any creative act...,70010000,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,70010100,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,10001674,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...,20001337.0,Type of Artists Brush/Applicator,"Indicates, with reference to the product brand...",30008495.0,ARTISTS SCRIPT LINER BRUSH,NaN


In [7]:
cols_to_keep = [
    "SegmentTitle", "SegmentDefinition", 
    "FamilyTitle", "FamilyDefinition", 
    "ClassTitle", "ClassDefinition", 
    "BrickTitle", "BrickDefinition_Includes", "BrickDefinition_Excludes"
]

gpc_df = gpc_df[cols_to_keep]
gpc_df.head()


,SegmentTitle,SegmentDefinition,FamilyTitle,FamilyDefinition,ClassTitle,ClassDefinition,BrickTitle,BrickDefinition_Includes,BrickDefinition_Excludes
0,Arts/Crafts/Needlework,All products that are used to any creative act...,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...
1,Arts/Crafts/Needlework,All products that are used to any creative act...,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...
2,Arts/Crafts/Needlework,All products that are used to any creative act...,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...
3,Arts/Crafts/Needlework,All products that are used to any creative act...,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...
4,Arts/Crafts/Needlework,All products that are used to any creative act...,Arts/Crafts/Needlework Supplies,All products that are used to any creative act...,Artists Painting/Drawing Supplies,Includes any products that may be described/ob...,Artists Brushes/Applicators,Includes any products that can be described/ob...,Specifically excludes Household Paint Brushes ...


In [8]:
copy_to_sql(gpc_df, "GPC_Labels", "demo_user", if_exists="replace")

In [ ]:
gpc_df["Segment"] = gpc_df["SegmentTitle"].astype(str) + " - " + gpc_df["SegmentDefinition"].astype(str)
gpc_df["Family"]  = gpc_df["FamilyTitle"].astype(str)   + " - " + gpc_df["FamilyDefinition"].astype(str)
gpc_df["Class"]   = gpc_df["ClassTitle"].astype(str)    + " - " + gpc_df["ClassDefinition"].astype(str)
gpc_df["Brick"]   = gpc_df["BrickTitle"].astype(str)    + " - " + gpc_df["BrickDefinition_Includes"].astype(str)

# Keep only the new concatenated columns
gpc_df = gpc_df[["Segment", "Family", "Class", "Brick"]]

In [10]:
gpc_df.head()

,Segment,Family,Class,Brick
0,Arts/Crafts/Needlework - All products that are...,Arts/Crafts/Needlework Supplies - All products...,Artists Painting/Drawing Supplies - Includes a...,Artists Brushes/Applicators - Includes any pro...
1,Arts/Crafts/Needlework - All products that are...,Arts/Crafts/Needlework Supplies - All products...,Artists Painting/Drawing Supplies - Includes a...,Artists Brushes/Applicators - Includes any pro...
2,Arts/Crafts/Needlework - All products that are...,Arts/Crafts/Needlework Supplies - All products...,Artists Painting/Drawing Supplies - Includes a...,Artists Brushes/Applicators - Includes any pro...
3,Arts/Crafts/Needlework - All products that are...,Arts/Crafts/Needlework Supplies - All products...,Artists Painting/Drawing Supplies - Includes a...,Artists Brushes/Applicators - Includes any pro...
4,Arts/Crafts/Needlework - All products that are...,Arts/Crafts/Needlework Supplies - All products...,Artists Painting/Drawing Supplies - Includes a...,Artists Brushes/Applicators - Includes any pro...


In [11]:
def load_gpc():
    gpc_df = pd.read_excel(GPC_PATH)
    cols_to_keep = [
    "SegmentTitle", "SegmentDefinition", 
    "FamilyTitle", "FamilyDefinition", 
    "ClassTitle", "ClassDefinition", 
    "BrickTitle", "BrickDefinition_Includes"
    ]

    gpc_df = gpc_df[cols_to_keep]

    return gpc_df

In [14]:
train_df = pd.read_csv('../data/correctly_matched_mapped_gpc.csv')
test2_df = pd.read_csv('../data/validated_actually_labeled_test_dataset.csv')
gpc_df = load_gpc()

segment_map = (gpc_df["SegmentTitle"].astype(str) + " - " + gpc_df["SegmentDefinition"].astype(str)).to_dict()
family_map  = (gpc_df["FamilyTitle"].astype(str)  + " - " + gpc_df["FamilyDefinition"].astype(str)).to_dict()
class_map   = (gpc_df["ClassTitle"].astype(str)   + " - " + gpc_df["ClassDefinition"].astype(str)).to_dict()
brick_map   = (gpc_df["BrickTitle"].astype(str)   + " - " + gpc_df["BrickDefinition_Includes"].astype(str)).to_dict()

if "segment" in train_df.columns:
    train_df["segment"] = train_df["segment"].map(segment_map).fillna(train_df["segment"])
if "family" in train_df.columns:
    train_df["family"] = train_df["family"].map(family_map).fillna(train_df["family"])
if "class" in train_df.columns:
    train_df["class"] = train_df["class"].map(class_map).fillna(train_df["class"])
if "brick" in train_df.columns:
    train_df["brick"] = train_df["brick"].map(brick_map).fillna(train_df["brick"])

if "predicted_segment" in test2_df.columns:
    test2_df["predicted_segment"] = test2_df["predicted_segment"].map(segment_map).fillna(test2_df["predicted_segment"])
if "predicted_family" in test2_df.columns:
    test2_df["predicted_family"] = test2_df["predicted_family"].map(family_map).fillna(test2_df["predicted_family"])
if "predicted_class" in test2_df.columns:
    test2_df["predicted_class"] = test2_df["predicted_class"].map(class_map).fillna(test2_df["predicted_class"])
if "predicted_brick" in test2_df.columns:
    test2_df["predicted_brick"] = test2_df["predicted_brick"].map(brick_map).fillna(test2_df["predicted_brick"])

In [15]:
train_df.head()

,new_id,id,product_name,segment,family,class,brick
0,1,0,WESSON Vegetable Oil 1 GAL,Food/Beverage,Oils/Fats Edible,Oils/Fats Edible,Oils/Fats Edible - Liquid
1,2,2,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,Food/Beverage,Prepared/Preserved Foods,Prepared/Preserved Foods - Liquid/Sauce/Soup,Soup - Ready to Eat (Shelf Stable)
2,3,3,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,Food/Beverage,Prepared/Preserved Foods,Prepared/Preserved Foods - Liquid/Sauce/Soup,Soup - Ready to Eat (Shelf Stable)
3,4,10,CAMPBELL'S SOUP BEAN AND HAM,Food/Beverage,Prepared/Preserved Foods,Prepared/Preserved Foods - Liquid/Sauce/Soup,Soup - Ready to Eat (Shelf Stable)
4,5,12,PREGO SAUCES TOMATO BASIL,Food/Beverage,Prepared/Preserved Foods,Prepared/Preserved Foods - Liquid/Sauce/Soup,Sauce - Ready to Eat (Shelf Stable)


In [ ]:
test2_df.head()